In [12]:
#THIS CODE IS NOT WORK AT THE MOMENT, NEED TO INVESTIGATE MORE IN THE FUTURE!!!
import numpy as np
from PIL import Image
import sys
sys.path.append("..")
from utils_image import cropLargestRecT
from scanimagetiffio import SITiffIO


In [13]:
def UnrotateCropFrame(Array, Angle, rotCenter): 
    """
    Unrotate each frame with the corresponding rotation angle and the rotation center 
    and crop the largest inner rectangle from the unrotated frame

    Args:
        Array (array): a 3D array of frames
        Angle (list): a list of rotation angles
        rotCenter (list): the rotation center from the center detection module
    """
    
    #for each frame, unrotate it with the corresponding rotation angle using Image.rotate
    #and store in an array
    NewFrames = []
    
    for i in range(Array.shape[0]):
        frame = Array[i,:,:]
        #unrotate the frame
        unrotatedFrame = Image.fromarray(frame).rotate(Angle[i], center=rotCenter)
        #crop the largest inner rectangle from the unrotated frame
        croppedFrame = cropLargestRecT(unrotatedFrame, rotCenter)
        #convert the PIL image back to int16
        croppedFrame = np.array(croppedFrame, dtype=np.int16)
        NewFrames.append(croppedFrame)      
        
    #convert the list to UnrotatedFrames to array
    NewFrames = np.array(NewFrames)
    
    return NewFrames

In [7]:
datafolder = '/home/zilong/Desktop/2D2P/Data/183_03082023'

# read the rotation center from the circlecenter txt file
circlecenterfilename = datafolder + "/DP_exp/circlecenter.txt"
with open(circlecenterfilename, "r") as f:
    # read the last row
    last_line = f.readlines()[-1]
    # assign the x and y coordinates to self.rotx and self.roty
    rotx = float(last_line.split()[0])
    roty = float(last_line.split()[1])
rotCenter = [rotx, roty]

S = SITiffIO()
S.open_tiff_file(datafolder+'/03082023_00004.tif', "r")
S.open_rotary_file(datafolder+'/REdata_20230803_173547.00004.txt')
#extract the last self.numFrames frames from the tiff file
tailArray, tailAng = S.tail(1000)

Loading rotary encoder file: /home/zilong/Desktop/2D2P/Data/183_03082023/REdata_20230803_173547.00004.txt
Calculating rotations and times from rotary encoder data...
Finished calculating rotations and times.
The rotary encoder file has 33177 timestamps in it.
Counted 9000 frames


In [33]:
tailArray, tailAng = S.tail(5000)

Counted 9000 frames


In [34]:
frames = UnrotateCropFrame(tailArray, tailAng, rotCenter)

In [35]:
#I want to speed up the UnrotateCropFrame by using parallel computing
from multiprocessing import Pool, cpu_count
from functools import partial

def process_frame(frame, angle, rot_center):
    unrotated_frame = Image.fromarray(frame).rotate(angle, center=rot_center)
    cropped_frame = cropLargestRecT(unrotated_frame, rot_center)
    cropped_frame = np.array(cropped_frame, dtype=np.int16)
    return cropped_frame

def UnrotateCropFrame_mp(Array, Angle, rotCenter):
    """
    Unrotate each frame with the corresponding rotation angle and the rotation center
    and crop the largest inner rectangle from the unrotated frame

    Args:
        Array (array): a 3D array of frames
        Angle (list): a list of rotation angles
        rotCenter (list): the rotation center from the center detection module
    """

    # Define the number of worker processes to use
    num_workers = cpu_count()

    # Create a partial function with the fixed arguments
    process_frame_partial = partial(process_frame, rot_center=rotCenter)

    # Use multiprocessing Pool to parallelize the computation
    with Pool(num_workers) as pool:
        # Call the helper function for each frame and get the results
        new_frames = pool.starmap(process_frame_partial, zip(Array, Angle))

    # Convert the list of frames to numpy array
    new_frames = np.array(new_frames)

    return new_frames


In [36]:
frames_mp = UnrotateCropFrame_mp(tailArray, tailAng, rotCenter)